In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR100
from torchvision import transforms

In [2]:
class CNNCifar100(nn.Module):
    def __init__(self):
        super(CNNCifar100, self).__init__()

        self.encoder = nn.Sequential(
            self._conv(3, 32),
            self._conv(32, 64),
            self._conv(64, 128),
        )

        self.q_mean = self._linear(2048, 128, relu=False)
        self.q_logvar = self._linear(2048, 128, relu=False)

        self.project = self._linear(128, 2048, relu=False)

        self.decoder = nn.Sequential(
			self._deconv(128, 64),
            self._deconv(64, 32),
            self._deconv(32, 3),
            nn.Sigmoid()
        )

    def forward(self, x):
        enc = self.encoder(x)
        unrolled = enc.view(-1, 2048)
        mu, lv = self.q_mean(unrolled), self.q_logvar(unrolled)
        rec_lat = self.project(self.reparam(mu, lv))
        dec = self.decoder(rec_lat.view(-1, 128, 4, 4))
        
        return dec, mu, lv

    def reparam(self, mu, lv):
        if self.training:
            std = torch.exp(0.5 * lv)
            eps = torch.randn(lv.size()).to(lv.device)
            return mu + std * eps
        else:
            return mu

    def _conv(self, channel_size, kernel_num):
        return nn.Sequential(
            nn.Conv2d(
                channel_size, kernel_num,
                kernel_size=4, stride=2, padding=1,
            ),
            nn.BatchNorm2d(kernel_num),
            nn.ReLU(),
        )

    def _deconv(self, channel_num, kernel_num):
        return nn.Sequential(
            nn.ConvTranspose2d(
                channel_num, kernel_num,
                kernel_size=4, stride=2, padding=1,
            ),
            nn.BatchNorm2d(kernel_num),
            nn.ReLU(),
        )

    def _linear(self, in_size, out_size, relu=True):
        return nn.Sequential(
            nn.Linear(in_size, out_size),
            nn.ReLU(),
        ) if relu else nn.Linear(in_size, out_size)

In [3]:
def gaussian_kls(mu, logvar, mean=False):

    klds = -0.5*(1 + logvar - mu.pow(2) - logvar.exp())

    if mean:
        reduce = lambda x: torch.mean(x, 1)
    else:
        reduce = lambda x: torch.sum(x, 1)

    total_kld = reduce(klds).mean(0, True)
    dimension_wise_kld = klds.mean(0)
    mean_kld = reduce(klds).mean(0, True)

    return total_kld, dimension_wise_kld, mean_kld

In [4]:
transform = transforms.Compose(
    [
        transforms.ToTensor(), 
    ]
)

trainset = CIFAR100(root='~/data', train=True, download=True, transform=transform)
train_dl = DataLoader(trainset, batch_size=2048, shuffle=True, num_workers=2)
testset = CIFAR100(root='~/data', train=False, download=True, transform=transform)
test_dl = DataLoader(testset, batch_size=2048, shuffle=False, num_workers=2)

print(len(train_dl), len(test_dl))

Files already downloaded and verified
Files already downloaded and verified
25 5


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)

model = CNNCifar100().to(device)

optimizer = Adam(params = model.parameters(), lr = 0.0001)

epochs = 100

scheduler = CosineAnnealingLR(optimizer, epochs*len(train_dl))

loss_fn = nn.MSELoss()

In [6]:
for epoch in range(epochs):
    model.train()
    tr_total_loss = 0
    for train_img, _ in tqdm(train_dl):
        train_img = train_img.to(device)

        gen_img, train_mu, train_lv = model(train_img)
        train_rec_loss = loss_fn(gen_img, train_img)
        train_kl_loss, _, _ = gaussian_kls(train_mu, train_lv)
        train_loss = train_rec_loss + train_kl_loss

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        scheduler.step()

        tr_total_loss += train_loss.item()

    model.eval()
    with torch.no_grad():
        va_total_loss = 0
        for valid_img, _ in tqdm(test_dl):
            valid_img = valid_img.to(device)

            gen_img, valid_mu, valid_lv = model(valid_img)
            valid_rec_loss = loss_fn(gen_img, valid_img)
            valid_kl_loss, _, _ = gaussian_kls(valid_mu, valid_lv)
            valid_loss = valid_rec_loss + valid_kl_loss

            va_total_loss += valid_loss.item()

    print(f"Epoch: {epoch} - TrainLoss: {tr_total_loss/len(train_dl)} - ValidLoss: {va_total_loss/len(test_dl)}")

100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.93it/s]


Epoch: 0 - TrainLoss: 7.938593959808349 - ValidLoss: 2.875922203063965


100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.98it/s]


Epoch: 1 - TrainLoss: 2.995992774963379 - ValidLoss: 2.57405571937561


100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.84it/s]


...

Epoch: 72 - TrainLoss: 1.0099696397781373 - ValidLoss: 1.0200442552566529


100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.02it/s]


Epoch: 73 - TrainLoss: 1.0110068726539612 - ValidLoss: 1.019699716567993


 60%|████████████████████████████████████████▊                           | 15/25 [00:02<00:01,  6.38it/s]


KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
    for test_case_idx in tqdm(range(10)):
        valid_img, _ = testset[test_case_idx]
        valid_img = valid_img.unsqueeze(dim=0).to(device)

        gen_img, _, _ = model(valid_img)        

        f, axarr = plt.subplots(1, 2)
        axarr[0].imshow(valid_img[0].cpu().permute(1, -1, 0).numpy())
        axarr[1].imshow(gen_img[0].cpu().permute(1, -1, 0).numpy())